<a href="https://colab.research.google.com/github/ML4SCI/DeepLearnHackathon/blob/main/HiggsBosonClassificationChallenge/higgs_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Higgs Classification Challenge

**Background:** High-energy collisions at the Large Hadron Collider (LHC) <br> produce particles that interact with particle detectors. One important task is <br>
to classify different types of collisions based on their physics content,<br> allowing physicists to find patterns in the data and to potentially unravel new <br> discoveries.

**Problem statement:** The discovery of the Higgs boson by CMS and ATLAS <br>
Collaborations was announced at CERN in 2012. In this challenge, we will use <br>
machine learning to classify events containing Higgs bosons from the background <br>
events which do not contain Higgs bosons.

**Dataset:** The dataset is hosted by the Center for Machine Learning  <br>
and Intelligent Systems at University of California, Irvine. <br>
The dataset can be found on the [UCI Machine learning Repository](https://archive.ics.uci.edu/ml/datasets/HIGGS)

**Description:** The dataset consists of a total of 11 million labeled samples <br>
of Higgs and background events produced by Monte Carlo simulations. Each sample <br>
consists of 28 features. The first 21 features are kinematic properties <br>
of the events. The last seven are functions of the first 21. The data labels <br>
are 1 for signal (an event with Higgs bosons) and 0 for background (an event <br>
without Higgs bosons).

**Steps to load the training dataset**   
If you are having problems with this part in Colab, you can also download the file manually and put it in your Google Drive. You can then [connect your Google Drive to Colab](https://towardsdatascience.com/different-ways-to-connect-google-drive-to-a-google-colab-notebook-pt-1-de03433d2f7a)

1. Download the dataset from the UCI website.

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00280/HIGGS.csv.gz

2. Unzip the dataset folder

In [ ]:
!gzip -d HIGGS.csv.gz

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
np.random.seed(1337)  # for reproducibility
import matplotlib.pyplot as plt

**Load the file using pandas library**

In [ ]:
data = pd.read_csv('./HIGGS.csv', header=None)

The first column is the labels (y). The other columns are all of our inputs (X).

The above dataset is a pandas dataframe. We can access the data using iloc. <br>
After that, we can turn it into a numpy array if we want or leave it as a <br>
pandas dataframe. **Use whatever you feel most comfortable with**.


In [ ]:
X = data.iloc[:,1:]
y = data.iloc[:,0]
#X = X.to_numpy(dtype=float) #Convert pandas dataframe to numpy array (optional)
#y = y.to_numpy(dtype=int)   #Convert pandas dataframe to numpy array (optional)

In [ ]:
print(X.shape)

To generate the following examples we used a smaller dataset containing only <br>
10,000 events. You may want to do something similar while getting your code <br>
set up but you should eventually use the full dataset.

**For final hackathon task submissions you must use the full test set.**

In [ ]:
X = X[:11000]
y = y[:11000]

In [ ]:
plt.hist(X.iloc[:,0], bins=30)
plt.title("lepton pT")
plt.xlabel("lepton pT")
plt.ylabel("number of events")
plt.show()

Next we can split our data into 9mil training data, 1mil validation data, 1mil <br>
test data.

For the rest of this hackathon, use `X_train`, `X_val`, `X_test` as input <br>
data and `y_train`, `y_val`, `y_test` as output data.

In [ ]:
X_train, X_val1, y_train, y_val1 = train_test_split(X, y, test_size=0.0909090909, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val1, y_val1, test_size=0.5, random_state=42)

In [ ]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

## **REMINDER: Use the Higgs dataset provided above for the Hackathon**

## Hackathon Task 1:

Data: `X_train`

Generate histograms of the different variables in `X_train` with proper axis <br>
labels and titles.

Detailed information on what each feature column is can be found in <br> *Attribute Information* section on the [UCI Machine learning Repository](https://archive.ics.uci.edu/ml/datasets/HIGGS). <br>
For further information, refer to the [paper](https://www.nature.com/articles/ncomms5308) by Baldi et. al

**Hint:** The first item is lepton pT.

The following may be helpful:

`names = ["lepton pT", "lepton eta", "lepton phi", "missing energy magnitude",` <br>
`"missing energy phi", "jet 1 pt", "jet 1 eta", "jet 1 phi", "jet 1 b-tag",` <br>
`"jet 2 pt", "jet 2 eta","jet 2 phi", "jet 2 b-tag", "jet 3 pt", "jet 3 eta",` <br>
`"jet 3 phi", "jet 3 b-tag", "jet 4 pt", "jet 4 eta", "jet 4 phi", "jet 4 b-tag",`<br>` "m_jj", "m_jjj", "m_lv", "m_jlv", "m_bb", "m_wbb", "m_wwbb"]`

`for index, name in enumerate(names):`

## Hackathon Task 2:

Data: `X_train`, `y_train`, `X_val`, `y_val`

Train a model by fitting it to the training data. Use at least one metric <br>
such as roc_auc_score, accuracy, etc. to analyze the model's performance on the <br>
validation data. Using that performance metric, optimize or improve your model. <br>
It should be clear from your notebook how you perform this optimization.

## Hackathon Task 3:

Data: `X_test`, `y_test`

**Note: The test data should be used only for final performance evaluation.** <br>
**Validation data can be used to tune your model but test data should not be** <br>
**used for model tuning.**

Without having done any optimization using the testing data set, analyze the <br>
performance of the model on the testing data. Your analysis should include <br> [roc_auc_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html), a ROC curve plot, and at least one other plot of your choice<br>
such as precision-recall curves, confusion matrix, etc.


# Deliverables:

-Fill out the pre- and post- hackathon surveys.
**Reminder: The hackathon tasks should be done using the Higgs dataset.** <br>
A pdf of the notebook with all three hackathon tasks completed. <br>
A copy of your colab/jupyter notebook (.ipynb and pdf) with all three hackathon tasks completed. <br>


File name convention: Try to give your submission files descriptive names, e.g. "Higgs_Yourname.pdf" and  <br>
"Higgs_Yourname.ipynb".


# Examples

The examples below use a different dataset (breast cancer diagnosis dataset) than what is provided above. Please use the Higgs data set for the hackathon.

Note: The following examples are meant to provide a starting point. You are encouraged to get creative. Feel free to look back to earlier assignments for inspiration and code examples.

## Decision Tree Example

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [ ]:
dataset = load_breast_cancer() #Use the Higgs dataset for the hackathon
X = dataset["data"]
y = dataset["target"]

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
classifier = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1),
    n_estimators=200,
    random_state=42
)

In [ ]:
classifier.fit(X_train, y_train)

In [ ]:
predictions = classifier.predict(X_test)

In [ ]:
y_hat = classifier.predict_proba(X_test)[:, 1]

In [ ]:
confusion_matrix(y_test, predictions)

In [ ]:
accuracy_score(y_test, predictions)

In [ ]:
from sklearn.metrics import roc_curve, auc

In [ ]:
def plot_roc_curve(y_test, y_hat):
    # Calculate ROC curve
    fpr, tpr, thresholds = roc_curve(y_test, y_hat)
    roc_auc = auc(fpr, tpr)
    # Plot the ROC curve
    plt.figure()
    plt.plot(fpr, tpr, label='ROC curve (area = %0.3f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--', label='Random Guessing')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve for Breast Cancer Classification')
    plt.legend()
    plt.show()
    print("AUC:", roc_auc)

In [ ]:
plot_roc_curve(y_test, y_hat)

## Basic Neural Network Example

In [ ]:
from numpy import loadtxt
from torch import nn
import torch
from sklearn.datasets import load_breast_cancer
from torch.utils.data import Dataset, DataLoader
import sklearn.preprocessing

In [ ]:
model_nn = nn.Sequential()
model_nn.append(nn.Linear(30, 64))
model_nn.append(nn.ReLU())
model_nn.append(nn.Linear(64, 8))
model_nn.append(nn.ReLU())
model_nn.append(nn.Linear(8, 1))
model_nn.append(nn.Flatten(start_dim=0))

In [ ]:
dataset = load_breast_cancer() #Use the Higgs dataset for the hackathon
X = dataset["data"]
y = dataset["target"]

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
# Note: The Higgs dataset is already pre-scaled so this step is not necessary
# in the actual hackathon
scaler = sklearn.preprocessing.StandardScaler()
scaler = scaler.fit(X)
X = scaler.transform(X)

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
X_train, X_test1, y_train, y_test1 = train_test_split(X, y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test1, y_test1, test_size=0.5, random_state=42)

In [ ]:
class PytorchDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X.copy()).float()
        self.y = torch.from_numpy(y.copy()).float()
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
train_data = PytorchDataset(X_train, y_train)
val_data = PytorchDataset(X_val, y_val)
test_data = PytorchDataset(X_test, y_test)

train_loader = DataLoader(train_data, batch_size=5, shuffle=True)
test_loader = DataLoader(test_data, batch_size=5, shuffle=False)
val_loader = DataLoader(val_data, batch_size=5, shuffle=False)

In [ ]:
def train_and_validate(train_loader, val_loader, model, optimizer, criterion, metric, num_epochs):
    history = {
        'epoch': [],
        'train_loss': [],
        'train_metric': [],
        'val_loss': [],
        'val_metric': []
    }  # Initialize a dictionary to store epoch-wise results

    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        epoch_loss = 0.0  # Initialize the epoch loss and metric values
        epoch_metric = 0.0

        # Training loop
        for X, y in train_loader:
            optimizer.zero_grad()  # Clear existing gradients
            outputs = model(X)  # Make predictions
            loss = criterion(outputs, y)  # Compute the loss
            loss.backward()  # Compute gradients
            optimizer.step()  # Update model parameters

            epoch_loss += loss.item()
            epoch_metric += metric(outputs, y)

        # Average training loss and metric
        epoch_loss /= len(train_loader)
        epoch_metric /= len(train_loader)

        # Validation loop
        model.eval()  # Set the model to evaluation mode
        with torch.no_grad():  # Disable gradient calculation
            val_loss = 0.0
            val_metric = 0.0
            for X_val, y_val in val_loader:
                outputs_val = model(X_val)  # Make predictions
                val_loss += criterion(outputs_val, y_val).item()  # Compute loss
                val_metric += metric(outputs_val, y_val)

            val_loss /= len(val_loader)
            val_metric /= len(val_loader)

        # Append epoch results to history
        history['epoch'].append(epoch_loss)
        history['train_loss'].append(epoch_loss)
        history['train_metric'].append(epoch_metric)
        history['val_loss'].append(val_loss)
        history['val_metric'].append(val_metric)

        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, '
              f'Train Metric: {epoch_metric:.4f}, Val Loss: {val_loss:.4f}, '
              f'Val Metric: {val_metric:.4f}')

    return history, model

In [ ]:
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_nn.parameters(), lr=0.001)

def accuracy_metric(target, pred):
    target = target.sigmoid().round()
    return torch.sum(pred == target).item() / len(pred)

In [ ]:
history, model_nn = train_and_validate(train_loader, val_loader, model_nn,
                                       optimizer=optimizer, criterion=criterion,
                                       metric=accuracy_metric, num_epochs=20)

In [ ]:
predictions = model_nn(test_loader.dataset.X).detach().numpy()

In [ ]:
def plot_roc_curve(y_test, y_hat):
    # Calculate ROC curve
    fpr, tpr, thresholds = roc_curve(y_test, y_hat)
    roc_auc = auc(fpr, tpr)
    # Plot the ROC curve
    plt.figure()
    plt.plot(fpr, tpr, label='ROC curve (area = %0.3f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--', label='Random Guessing')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve for Breast Cancer Classification')
    plt.legend()
    plt.show()
    print("AUC:", roc_auc)

In [ ]:
plot_roc_curve(y_test, predictions)

## Saving objects to drive

In [ ]:
# Connecting notebook to google drive
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# Saving serializable decision tree object
import pickle as pkl
with open("/content/drive/MyDrive/classifier.pkl", "wb") as f:
    pkl.dump(classifier, f)

In [ ]:
# Loading serializable decision tree object
with open("/content/drive/MyDrive/classifier.pkl", "rb") as f:
    new_classifier = pkl.load(f)

In [ ]:
# Saving neural network model weights using pytorch
torch.save(model_nn.state_dict(), "/content/drive/MyDrive/my_pytorch_model.h5")##Saving model weights

In [ ]:
# Loading neural network model weights using pytorch
new_model_nn = nn.Sequential()
new_model_nn.append(nn.Linear(30, 64))
new_model_nn.append(nn.ReLU())
new_model_nn.append(nn.Linear(64, 8))
new_model_nn.append(nn.ReLU())
new_model_nn.append(nn.Linear(8, 1))
new_model_nn.append(nn.Flatten(start_dim=0))

new_model_nn.load_state_dict(torch.load("/content/drive/MyDrive/my_pytorch_model.h5"))

In [ ]:
# Saving numpy array of predictions
np.save("/content/drive/MyDrive/predictions.npy", predictions)

In [ ]:
# Loading numpy array of predictions
saved_predictions = np.load("/content/drive/MyDrive/predictions.npy")